### Benson Team Awesome

C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


|Code      | explanation                                                                                     |
|:-------- |:----------------------------------------------------------------------------------------------- | 
| C/A      | Control Area (A002)                                                                          | 
| UNIT     | Remote Unit for a station (R051)                                                             | 
| SCP      | Subunit Channel Position represents an specific address for a device (02-00-00)              | 
| STATION  | Represents the station name the device is located at                                         | 
| LINENAME | Represents all train lines that can be boarded at this station                               | 
|          |   Normally lines are represented by one character.  LINENAME 456NQR repersents train server for | 
|          |   4, 5, 6, N, Q, and R trains.                                                                  | 
| DIVISION | Represents the Line originally the station belonged to BMT, IRT, or IND                      | 
| DATE     | Represents the date (MM-DD-YY)                                                               |   
| TIME     | Represents the time (hh:mm:ss) for a scheduled audit event                                   | 
| DESc     | Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)                | 
|          |   1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.         | 
|          |   2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. |
|ENTRIES   | The comulative entry register value for a device|
|EXIST     | The cumulative exit register value for a device|


In [39]:
import csv
import urllib
import io
import pandas as pd
import numpy as np
import multiprocessing as mlp
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160326.txt')

In [40]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248


In [41]:
data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [42]:
data.rename(columns={data.columns[-1]: data.columns[-1].replace(' ','')}, inplace=True)
data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [43]:
#data[mask, 'Delta'] = abs((data[mask,'ENTRIES'] - data[mask,'ENTRIES'].shift(-1)) + (data[mask,'EXITS'] - data[mask,'EXITS'].shift(-1)))
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248


In [ ]:
#test = data.sort_values(['Delta'])


In [ ]:
#data[['TIME','DATE']] = pd.to_datetime(data[['TIME','DATE']])


In [ ]:
#df_night = data[data['TIME' ==  ]

In [44]:
#Create the keys that is used to make a mask

mask_key = data.iloc[:, 0].copy()
for j in range(1,4):
        mask_key += ',' + data.iloc[:,j]
        
#print the first tree lines to have an idea of it
print(mask_key[:3])

#drop the keys that is 
mask_key.drop_duplicates(inplace=True)
print(data.head())

0    A002,R051,02-00-00,59 ST
1    A002,R051,02-00-00,59 ST
2    A002,R051,02-00-00,59 ST
Name: C/A, dtype: object
    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  00:00:00   
1  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  04:00:00   
2  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  08:00:00   
3  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  12:00:00   
4  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  16:00:00   

      DESC  ENTRIES    EXITS  
0  REGULAR  5590801  1889027  
1  REGULAR  5590828  1889032  
2  REGULAR  5590852  1889059  
3  REGULAR  5590958  1889162  
4  REGULAR  5591207  1889248  


In [94]:
#Initialize the Delta columns as zeros
data['Delta'] = np.zeros(data.shape[0])

#temporary function to fill in the deltas by selections
def fill_in(mask):
    data.loc[mask,'Delta'] = abs((data.loc[mask, 'ENTRIES'] - data.loc[mask, 'ENTRIES'].shift(-1)) + (data.loc[mask, 'EXITS'] - data.loc[mask, 'EXITS'].shift(-1)))

In [ ]:
for i in range(mask_key.shape[0]):
    s = mask_key.iloc[i].split(',')
    mask = (data.iloc[:,0] == s[0])  & (data.iloc[:,1] == s[1]) & (data.iloc[:,2] == s[2]) & (data.iloc[:,3] == s[3])
    fill_in(mask)

['A002', 'R051', '02-00-00', '59 ST']
    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  00:00:00   
1  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  04:00:00   
2  A002  R051  02-00-00   59 ST   NQR456      BMT  03/19/2016  08:00:00   

      DESC  ENTRIES    EXITS  Delta  
0  REGULAR  5590801  1889027      0  
1  REGULAR  5590828  1889032      0  
2  REGULAR  5590852  1889059      0  
0          True
1          True
2          True
3          True
4          True
5          True
6          True
7          True
8          True
9          True
10         True
11         True
12         True
13         True
14         True
15         True
16         True
17         True
18         True
19         True
20         True
21         True
22         True
23         True
24         True
25         True
26         True
27         True
28         True
29         True
          ...  
192916    Fal

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Delta
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027,0
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032,0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059,0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162,0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248,0


In [96]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Delta
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,00:00:00,REGULAR,5590801,1889027,32
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,04:00:00,REGULAR,5590828,1889032,51
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,08:00:00,REGULAR,5590852,1889059,209
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,12:00:00,REGULAR,5590958,1889162,335
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/19/2016,16:00:00,REGULAR,5591207,1889248,406


In [31]:
data.ix[:,0:4] == s.split(',')[0]

['A002', 'R051', '02-00-00', '59 ST']

In [77]:
list(mask_key)

['A002,R051,02-00-00,59 ST',
 'A002,R051,02-00-01,59 ST',
 'A002,R051,02-03-00,59 ST',
 'A002,R051,02-03-01,59 ST',
 'A002,R051,02-03-02,59 ST',
 'A002,R051,02-03-03,59 ST',
 'A002,R051,02-03-04,59 ST',
 'A002,R051,02-03-05,59 ST',
 'A002,R051,02-03-06,59 ST',
 'A002,R051,02-05-00,59 ST',
 'A002,R051,02-05-01,59 ST',
 'A002,R051,02-06-00,59 ST',
 'A006,R079,00-00-00,5 AV/59 ST',
 'A006,R079,00-00-01,5 AV/59 ST',
 'A006,R079,00-00-02,5 AV/59 ST',
 'A006,R079,00-00-03,5 AV/59 ST',
 'A006,R079,00-00-04,5 AV/59 ST',
 'A006,R079,00-03-00,5 AV/59 ST',
 'A006,R079,00-03-01,5 AV/59 ST',
 'A006,R079,00-03-02,5 AV/59 ST',
 'A007,R079,01-05-00,5 AV/59 ST',
 'A007,R079,01-05-01,5 AV/59 ST',
 'A007,R079,01-06-00,5 AV/59 ST',
 'A007,R079,01-06-01,5 AV/59 ST',
 'A007,R079,01-06-02,5 AV/59 ST',
 'A007,R079,01-06-03,5 AV/59 ST',
 'A010,R080,00-00-00,57 ST-7 AV',
 'A010,R080,00-00-01,57 ST-7 AV',
 'A010,R080,00-00-02,57 ST-7 AV',
 'A010,R080,00-00-03,57 ST-7 AV',
 'A010,R080,00-00-04,57 ST-7 AV',
 'A010